In [1]:
from IPython.display import display, HTML
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as pyfunc
import pyspark.sql.types as pytype
display(HTML('<style>pre { white-space: pre !important; }</style>'))

## Read

In [2]:
spark = SparkSession.builder.getOrCreate()
spotify_df = spark.read.parquet("/workspaces/Introduction-to-Data-Science/ca3/spotify.parquet")
spotify_df.printSchema()

24/04/15 11:13:18 WARN Utils: Your hostname, codespaces-7af96c resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/04/15 11:13:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/15 11:13:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- album: string (nullable = true)
 |-- album_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- artist_ids: string (nullable = true)
 |-- track_number: long (nullable = true)
 |-- disc_number: long (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: long (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- year: long (nullable = true)
 |-- release_date: string (nullable = true)



## preprocess

In [3]:
spotify_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+-----------+-----------------+--------------------+------------------+------------------+------------------+-----------+--------------+----+------------+
|                  id|                name|               album|            album_id|             artists|          artist_ids|track_number|disc_number|explicit|      danceability|            energy|key|           loudness|mode|speechiness|     acousticness|    instrumentalness|          liveness|           valence|             tempo|duration_ms|time_signature|year|release_date|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+--------

In [11]:
spotify_df.select('disc_number').distinct().collect()


[Row(disc_number=7),
 Row(disc_number=6),
 Row(disc_number=9),
 Row(disc_number=5),
 Row(disc_number=1),
 Row(disc_number=10),
 Row(disc_number=3),
 Row(disc_number=12),
 Row(disc_number=8),
 Row(disc_number=11),
 Row(disc_number=2),
 Row(disc_number=4),
 Row(disc_number=13)]

In [7]:
df = spotify_df.withColumn("release_date", pyfunc.to_date(spotify_df.release_date, "yyyy-MM-dd"))
df = df.withColumn("artists", pyfunc.from_json(pyfunc.col("artists"), pytype.ArrayType(pytype.StringType())))
df = df.withColumn("artist_ids", pyfunc.from_json(pyfunc.col("artist_ids"), pytype.ArrayType(pytype.StringType())))
# df = df.drop('year')
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- album: string (nullable = true)
 |-- album_id: string (nullable = true)
 |-- artists: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- artist_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- track_number: long (nullable = true)
 |-- disc_number: long (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: long (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- year: long (nulla

In [8]:
first_row = df.filter(df.year == 2014).first()
print(first_row['artists'][0])
print(first_row['artists'][1])

Mauricio Kagel
Ensemble Musikfabrik


In [ ]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+-----------+-----------------+--------------------+------------------+------------------+------------------+-----------+--------------+----+------------+
|                  id|                name|               album|            album_id|             artists|          artist_ids|track_number|disc_number|explicit|      danceability|            energy|key|           loudness|mode|speechiness|     acousticness|    instrumentalness|          liveness|           valence|             tempo|duration_ms|time_signature|year|release_date|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+--------

In [ ]:
first_row = df.first()
print(first_row['artists'])

["['Rage Against The Machine']"]


24/04/15 10:31:30 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
first_row = spotify_df.filter(df.year == 2014).first()
print(first_row['artists'])

['Mauricio Kagel', 'Ensemble Musikfabrik']
